# Exploring groundwater data from Lizard

In this notebook, you will experiment how to use the `hydropandas` package to access, visualize and explore meta data from Lizard, a cloud datawarehouse that can be used to store groundwater observations. Data sources:
1. Vitens (default, public access): Vitens is the largest drinking water company in the Netherlands, and it has more than 10.000 groundwater wells and more than 50.000 timeseries in its datawarehouse. The data spans from the 1930’s to the present, and it is constantly updated with new observations. Vitens also validates the data using ArtDiver and provides quality flags and comments for each observation. The data is open to the public and you can find more information at https://vitens.lizard.net.
2. Others: If you have access to other Lizard data sources, you can specify the `organisation` parameter in the `lizard` functions to access those data sources. Currently, only "vitens" is officially supported. Note that you then may need to specify authentication as well by means of a personal API key.

---

Feel free to customize and expand upon this introduction as needed. Happy coding! 🚀🌍📊


## <a id=top></a>Notebook contents

1. [Find groundwater wells on a map](#extent)
2. [Analyse groundwater observations](#gwobs)
3. [Build a Pastas model](#pastas)

In [ ]:
import pastas as ps

import hydropandas as hpd

In [ ]:
hpd.util.get_color_logger("INFO")

The `vitens` data source is used by default, and does not require authentication. If you have access to other Lizard data sources, you can specify the `organisation` parameter in the `read_lizard` function to access those data sources. Currently, only "vitens" is officially supported.

For others than Vitens, you may need authentication. You can use the `auth` parameter to pass your credentials. According to the [Lizard documentation](https://docs.lizard.net/c_endpoints.html#authentication), you should create a personal API key at `https://{organisation}.lizard.net/management/` and enter that in the cell below. If you do not have access to other data sources, you can use `auth=None` to skip authentication.

In [ ]:
## Lizard uses the organisation parameter to specify the data source.
organisation = "vitens"
auth = None

## UNCOMMENT AND SET YOUR API KEY IF YOU NEED TO SPECIFY CREDENTIALS
# your_api_key = "your_api_key_here"
# auth = ("__key__", your_api_key)

## Get observations from extent <a id=extent></a>

Use `ObsCollection` to find monitoring wells by specifying a geographical extent in Rijksdriehoeks coordinates.

In [ ]:
my_extent = (137000, 138000, 458000, 459000)
oc = hpd.read_lizard(extent=my_extent, organisation=organisation, auth=auth)

Visualize all groundwater wells inside the extent on a map (visualize the `ObsCollection`). The markers are clickable to show a preview of the availables observations.

In [ ]:
oc.plots.interactive_map(color="red", zoom_start=15, tiles="Esri.WorldImagery")

Print all the retrieved groundwater wells and tubes, and make a plot of the observations.

In [ ]:
oc

In [ ]:
oc.plots.section_plot(plot_obs=True)

## Groundwater observations <a id=gwobs></a>

Now lets download the groundwater level observation using the `from_lizard` function of a `GroundwaterObs` object. The code below reads the groundwater level timeseries for the well UPWP016 from Lizard and makes a plot.

In [ ]:
gw_lizard = hpd.GroundwaterObs.from_lizard(
    "UPWP016",
    tmin="1900-01-01",
    tmax="2030-01-01",
    organisation="vitens",
    auth=auth,
)
print(gw_lizard)

ax = gw_lizard["value"].plot(
    figsize=(12, 5),
    marker=".",
    grid=True,
    label=gw_lizard.name,
    legend=True,
    xlabel="Date",
    ylabel="m NAP",
    title="Groundwater observations for " + gw_lizard.name,
)

The groundwater observations contain a validation flag per timestamp. These can 'betrouwbaar' (reliable), 'onbetrouwbaar' (unreliable) en 'onbeslist' (unvalidated). Below flags of the timeseries are shown as a percentage, and the unreliable timestamps are printed.

In [ ]:
print(gw_lizard["flag"].value_counts(normalize=True) * 100)
gw_lizard[gw_lizard["flag"] == "onbetrouwbaar"]

## Pastas model <a id=pastas></a>

Lets make a Pastas model for this groundwater well (starting from 2015) and use the nearest KNMI station for meteorological data

In [ ]:
# Get the precipitation and evaporation data from the KNMI
precipitation = hpd.PrecipitationObs.from_knmi(
    xy=(gw_lizard.x, gw_lizard.y),
    start=gw_lizard.index[0],
    end=gw_lizard.index[-1],
    fill_missing_obs=True,
)
evaporation = hpd.EvaporationObs.from_knmi(
    xy=(gw_lizard.x, gw_lizard.y),
    meteo_var="EV24",
    start=gw_lizard.index[0],
    end=gw_lizard.index[-1],
    fill_missing_obs=True,
)

# Create a Pastas Model
ml = ps.Model(gw_lizard["value"], name=gw_lizard.name)

# Add the recharge data as explanatory variable
ts1 = ps.RechargeModel(
    precipitation["RH"].resample("D").first(),
    evaporation["EV24"].resample("D").first(),
    ps.Gamma(),
    name="rainevap",
    settings=("prec", "evap"),
)

# Add the stressmodel to the model and solve for period after 2015
ml.add_stressmodel(ts1)
ml.solve(tmin="2015")
ml.plots.results(figsize=(10, 6))